# Discretization of a 1d grid
### Spatial grid
We discretize a one–dimensional spatial domain of length $L$ into $N$ uniformly
spaced grid points.

### 1D Uniform Grid Conventions
Let the continuous domain be an interval on the real line.  
We discretize it into $N$ points with spacing $\Delta x$.

Throughout, define:
- left endpoint $a$
- right endpoint $b$
- length $L=b-a$
- number of grid points $N$
- spacing $\Delta x$


## Left-closed, right-open interval
A left-closed, right-open interval is the set
$$
  x \in [a,b)
$$
which is equivalent
$$
  x \in X = \{\mathbb{R}:a \leq x < b\}
$$
which includes $a$ and excludes $b$.
##### Grid definition
$$
x_i = a + i\,\Delta x,
$$
where
$$\begin{gather}
  i=0,1,\dots,N-1 \\
  \Delta x = \frac{b-a}{N}
\end{gather}$$

On a uniform grid, the point is $x_{N-1} = b-\Delta x$

In [7]:
import numpy as np

N = 400
L = 1.0
a = -L/2
b = L/2
X = np.linspace(a, b, N, endpoint=False)
dx = x[1] - x[0]

assert a in X
assert b not in X
assert np.isclose(X[-1], b - dx)

## Left open, right-closed interval
$$
  x \in X = (a,b\,] = \{\mathbb{R}|a < x \leq b\}
$$
where
- $a \notin X$
- $b \in X$

In [8]:
import numpy as np

N = 400
L = 1.0
a = -L/2
b = L/2

dx = (b-a)/N
X = a + (np.arange(N) + 1)*dx

assert a not in X
assert b in X
assert np.isclose(X[0],a+dx)

### Open Interval, N interior points
For Dirichlet boundary problems where $\phi(a)$ and $\phi(b)$ are imposed separately and you only solve for interior values.

$$x\in(a,b) = X = \{\mathbb{R}:a < x < b\}$$

$$
  x_i = a + (i+1)\Delta x,\quad i=0,1,...,N-1
$$
where
$$\Delta x = \frac{b-a}{N+1}$$



In [9]:
import numpy as np

N = 400
L = 1.0
a = -L/2
b = L/2
dx = (b-a)/(N+1)
X = a + (np.arange(N) + 1)*dx

assert a not in X
assert b not in X


### Open interval, Midpoint centered grid
This is natural in finite-volume / conservative discretizations.
$$ x \in (a,b) = X = \{\mathbb{R}:a < x < b\}$$

$$
  x_i = a + \left(i+\frac{1}{2}\right)\Delta x, \quad i=0,1,...,N-1
$$
with
$$ \Delta x = \frac{b-a}{N}$$

In [10]:
N = 400
L = 1.0
a = -L/2
b = L/2

dx = (b-a)/N
X = a + (np.arange(N) + 0.5)*dx

assert a not in X
assert b not in X

### Closed interval
$$ x \in [a,b] = X = \{\mathbb{R}:a \leq x \leq b\} $$

$$
  x_i = a + i \Delta x, \quad i=0,1,\ldots,N-1
$$
where
$$
  \Delta x = \frac{b-a}{N-1}
$$

In [11]:
N = 400
L = 1.0
a = -L/2
b = L/2

X = np.linspace(a, b, N, endpoint=True)
dx = X[1] - X[0] 

assert np.isclose(dx, (b-a)/(N-1))
assert a in X
assert b in X


## Grid1D Class Design

In [ ]:
from dataclasses import dataclass
import numpy as np
from enum import Enum, auto

class GridType1D(Enum):
  CLOSED = auto()
  OPEN = auto()
  LEFT_CLOSED = auto()
  RIGHT_CLOSED = auto()
  MIDPOINT = auto()
  INTERIOR = auto()

@dataclass(frozen=True)
class Grid1d:
  a: float
  b: float
  N: int
  grid_type: GridType1D = GridType1D.CLOSED

  def __post_init__(self):
    if self.b <= self.a:
      raise ValueError("Require b > a")
    if self.N <= 0:
      raise ValueError("Require N > 0")
    
    @property
    def dx(self) -> float:
        if self.grid_type is GridType1D.CLOSED:
            return self.L / (self.N - 1)
        elif self.grid_type is GridType1D.INTERIOR:
            return self.L / (self.N + 1)
        else:
            return self.L / self.N
    
    @property
    def x(self) -> np.ndarray:
        dx = self.dx

        if self.grid_type is GridType1D.LEFT_CLOSED:
            return self.a + dx * np.arange(self.N)

        elif self.grid_type is GridType1D.RIGHT_CLOSED:
            return self.a + dx * (np.arange(self.N) + 1)

        elif self.grid_type is GridType1D.OPEN:
            return self.a + dx * (np.arange(self.N) + 1)

        elif self.grid_type is GridType1D.INTERIOR:
            return self.a + dx * (np.arange(self.N) + 1)

        elif self.grid_type is GridType1D.MIDPOINT:
            return self.a + dx * (np.arange(self.N) + 0.5)

        elif self.grid_type is GridType1D.CLOSED:
            return np.linspace(self.a, self.b, self.N)

        else:
            raise RuntimeError("Unhandled GridType1D")